Объединенный датасет [FIX MEне доступен по ссылке](*).

Положить в папку data содержимое так, чтобы были следующие пути:  
* \$(ROOT_DIR)/data/full-rtsd/...
* \$(ROOT_DIR)/data/full-gt.csv

> *gt_Set_NaN.csv - содержит тот же датасет, но значения колонки Set обнулено*

gt - датафрейм содержащий:  
* имена файлов - поле filename
* класс знака - поле sign_class
* координаты знаков
* в какой набор включен знак - поле Set $\in$ $\{train, valid, test\}$  

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (25,8)
import numpy as np
import random
import torch
from torch import nn
import seaborn as sns
import pandas as pd
import os
import pathlib
import shutil
import cv2
import PIL
import sys

try:
    import google.colab
    IN_COLAB = True
    from google.colab import drive
    drive.mount('/content/drive')
except:
    IN_COLAB = False

TEXT_COLOR = 'black'

# Зафиксируем состояние случайных чисел
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
if not IN_COLAB:
    %run utils.ipynb
    PROJECT_ROOT = pathlib.Path(os.path.join(os.curdir, os.pardir))
else:
    PROJECT_ROOT = pathlib.Path('')
    
DATA_DIR = PROJECT_ROOT / 'data'
NOTEBOOKS_DIR = PROJECT_ROOT / 'notebooks'

if (NOTEBOOKS_DIR / 'full-gt.csv').is_file():
    full_gt = pd.read_csv(NOTEBOOKS_DIR / 'full-gt.csv')
else:
    full_gt = pd.read_csv(DATA_DIR / 'full-gt.csv')

FORMATED_GT_PATH = "formated_full_gt.csv"
FULL_GT_SRC_LEN = len(full_gt.index)
display(full_gt)

In [ ]:
full_gt_unique_filenames = set(full_gt['filename'])
full_gt_unique_filenames_size = len(full_gt_unique_filenames)
%run utils.ipynb
import ast
import re

i = 0;

if os.path.isfile(FORMATED_GT_PATH):
    print("FORMATED GT EXIST. LOAD IT")
    formated_full_gt_df = pd.read_csv(FORMATED_GT_PATH, dtype=object)
    # display(formated_full_gt_df)
    formated_full_gt_df['coords'].replace({'\n ':',', ' \s+': ' ', '\[ ': '['}, regex=True, inplace=True)
    # display(formated_full_gt_df)
    formated_full_gt_df['coords'] = formated_full_gt_df['coords'].apply(
        lambda x: ast.literal_eval(x)
    )
    
    formated_full_gt_df['size'] = formated_full_gt_df['size'].apply(
        lambda x: ast.literal_eval(x)
    )
else:
    print("FORMATED GT DOESNT EXIST. CREATE IT")
    # get all original filenames
    full_gt_unique_filenames = set(full_gt['filename'])
    
    formated_full_gt_list = []

    import imagesize
    
    for src_filename_iterator in list(full_gt_unique_filenames):

        mask = np.in1d(full_gt['filename'], [src_filename_iterator])
        coord_data_arr = full_gt[mask][['x_from', 'y_from', 'width', 'height']].to_numpy()
        
        filepath = DATA_DIR / "rtsd-frames" / src_filename_iterator
        origW, origH = imagesize.get(filepath)
                
        rel_coord = []
        for coord in coord_data_arr:
            # make from x, y, dx, dx -> x1, y1, x2, y2
            CV2RectangleCoords = ConvertAbsTLWH2CV2Rectangle(coord)
   
            # make from x1, y1, x2, y2 -> x, y, w, h
            CV2CircleCoords = ConvertCV2Rectangle2CenterXYWH(CV2RectangleCoords)
            
            # make x, y, w, h -> relative x, y, w, h
            rel_instance = MakeRel(CV2CircleCoords, origW, origH)

            rel_coord.append(rel_instance)
            
        if i % 100 == 0:
            printProgressEnum(i, full_gt_unique_filenames_size)
        i += 1

        formated_full_gt_list.append([str(filepath), rel_coord, [origW, origH]])

    formated_full_gt_df = pd.DataFrame(formated_full_gt_list, columns=['filepath', 'coords', 'size'])
    formated_full_gt_df.to_csv("formated_full_gt.csv", index=False)

formated_full_gt_df

# simple test

In [ ]:
instance = formated_full_gt_df.iloc[15466]
print(instance)
img = cv2.imread(str(instance['filepath']))
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)


h, w = img.shape[0], img.shape[1]
print('Shape:', w, h)


for i in instance['coords']:
    
    xywh = UnmakeRel(i, w, h)
    x1y1x2y2 = ConvertCenterXYWH2CV2Rectangle(xywh)
    print('+', MakeRel(x1y1x2y2, w, h))
    print('xywh', xywh)
    print('x1y1x2y2', x1y1x2y2)
    
    
    img = cv2.rectangle(img, (x1y1x2y2[0], x1y1x2y2[1]), 
                        (x1y1x2y2[2], x1y1x2y2[3]), 
                        (255, 0, 0), 
                        3)
    
    img = cv2.circle(img, 
                     (xywh[0], xywh[1]), 
                     xywh[2] // 2, 
                     (255, 255, 0), 
                     3)

    
plt.imshow(img)
plt.show()

In [ ]:
if 'set' in formated_full_gt_df.columns:
    print('SET ALREADY EXIST')
else:
    print('SET DOESNT EXIST. LETS CREATE IT')
    formated_full_gt_df_index_count = len(formated_full_gt_df.index)
    TRAIN_SIZE = round(0.7 * formated_full_gt_df_index_count)
    VALID_SIZE = round(0.2 * formated_full_gt_df_index_count)
    TEST_SIZE = round(formated_full_gt_df_index_count - TRAIN_SIZE - VALID_SIZE)
    
    # print('assert:', TRAIN_SIZE + VALID_SIZE + TEST_SIZE, '==', formated_full_gt_df_index_count)
    
    assert TRAIN_SIZE + VALID_SIZE + TEST_SIZE == formated_full_gt_df_index_count, 'wrong split'
    set_series = pd.Series('test', index=range(TEST_SIZE)).append(
        pd.Series('train', index=range(TRAIN_SIZE)).append(
            pd.Series('valid', index=range(VALID_SIZE))
        )
    ).sample(frac=1).reset_index(drop=True)
    formated_full_gt_df['set'] = set_series
    display(formated_full_gt_df)
    formated_full_gt_df.to_csv("formated_full_gt.csv", index=False)
    
display(formated_full_gt_df)

Now we have pd.DataFrame that contains filenames, list of relative coordinates, corresponding photo resoulutions and marks for set. 

Let's make DataSets for it.

In [ ]:
import yaml
YOLO_MODEL_HOME_DIR = DATA_DIR / 'YoloV5'
AUGMENT_HOME_DIR = YOLO_MODEL_HOME_DIR / 'utils'

if YOLO_MODEL_HOME_DIR not in sys.path:
    sys.path.append(str(YOLO_MODEL_HOME_DIR))

from models.yolo import Model
from torch.optim import SGD, Adam, AdamW, lr_scheduler
from utils.augmentations import Albumentations, augment_hsv, copy_paste, letterbox, mixup, random_perspective

hyps_file = YOLO_MODEL_HOME_DIR / 'data/hyps' / "hyp.scratch.yaml"
with open(hyps_file, errors='ignore') as f:
    hyp = yaml.safe_load(f)

class CreateDataSet(torch.utils.data.Dataset):
    def __init__(self, df, set_label, img_size=640, batch_size=16, augment=False, hyp=None):
        
        self.img_size = img_size
        self.augment = augment
        self.hyp = hyp

        self.df = df[df['set']==set_label]
        self.albumentations = Albumentations() if augment else None
        
    def loadImage(self, instance):
        path, (w0, h0) = instance['filepath'], instance['size']
        im = cv2.imread(path)
        assert im is not None, f'Image Not Found {path}'
        
        r = self.img_size / max(h0, w0)  # ratio
        if r != 1:  # if sizes are not equal
            im = cv2.resize(im, (int(w0 * r), int(h0 * r)),
                            interpolation=cv2.INTER_AREA if r < 1 and not self.augment else cv2.INTER_LINEAR)
        return im, (h0, w0), im.shape[:2]
    
    def __getitem__(self, index):
        
        # locate img info from DataFrame
        instance = self.df.iloc[index]
        
        # get Img, src height, width and resized height, width
        img, (h0, w0), (h, w) = self.loadImage(instance)
        
        shape = self.img_size
        
        # make img square
        # print('>', (img>1).sum())
        # print('<=', (img<=1).sum())
        img, ratio, pad = letterbox(img, shape, auto=False, scaleup=self.augment)
        # print(pad)
        # store core shape info
        shapes = (h0, w0), ((h / h0, w / w0), pad)  # for COCO mAP rescaling
        
        # add class to labels. We have 1 class, so just add zeros into first column
        labels = np.array(instance['coords'])
        labels = np.c_[np.zeros(labels.shape[0]), labels]
        # print(labels)
        
        # fix labels location caused by letterbox
        labels[:, 1:] = xywhn2xyxy(labels[:, 1:], ratio[0] * w, ratio[1] * h, padw=pad[0], padh=pad[1])
        
        if self.augment:
            img, labels = random_perspective(img, labels,
                degrees=hyp['degrees'],
                translate=hyp['translate'],
                scale=hyp['scale'],
                shear=hyp['shear'],
                perspective=hyp['perspective'])
                
        labels[:, 1:5] = xyxy2xywhn(labels[:, 1:5], w=img.shape[1], h=img.shape[0], clip=False, eps=1E-3)
        
        # YOLO augmentation technique (!copy-paste!)
        if self.augment:
            # print('augm for', index, instance['filepath'])
            # Albumentations
            img, labels = self.albumentations(img, labels)
            nl = len(labels)  # update after albumentations

            # HSV color-space
            augment_hsv(img, hgain=hyp['hsv_h'], sgain=hyp['hsv_s'], vgain=hyp['hsv_v'])

            # Flip up-down
            if random.random() < hyp['flipud']:
                img = np.flipud(img)
                if nl:
                    labels[:, 2] = 1 - labels[:, 2]

            # Flip left-right
            if random.random() < hyp['fliplr']:
                img = np.fliplr(img)
                if nl:
                    labels[:, 1] = 1 - labels[:, 1]
                    
        nl = len(labels)
        
        # why out size (?, 6)?? 
        labels_out = torch.zeros((nl, 6))
        if nl:
            labels_out[:, 1:] = torch.from_numpy(labels)   

        img = img.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
        img = np.ascontiguousarray(img)
        
        return torch.from_numpy(img), labels_out, instance['filepath'], shapes
    
    def __len__(self):
        return len(self.df.index)
    
    @staticmethod
    def collate_fn(batch):
        img, label, path, shapes = zip(*batch)  # transposed
        for i, l in enumerate(label):
            l[:, 0] = i  # add target image index for build_targets()
        return torch.stack(img, 0), torch.cat(label, 0), path, shapes

In [ ]:
def createDataLoaderAndDataSet(df, set_label, imgsz, batch_size, hyp=None, augment=False,shuffle=True):
    
    
    from torch.utils.data import DataLoader
    
    dataset = CreateDataSet(df, set_label, img_size=imgsz, augment=augment)
    batch_size = min(batch_size, len(dataset))
    
    sampler = None #distributed.DistributedSampler(dataset, shuffle=shuffle)
    
    loader = DataLoader(dataset, # InfiniteDataLoader ?
                        batch_size=batch_size,
                        shuffle=shuffle and sampler is None,
                        # num_workers=nw,  # doesnt work in Windows
                        sampler=sampler,
                        pin_memory=True,
                        collate_fn=CreateDataSet.collate_fn)

    
    return loader, dataset

IMG_SIZE = 1280
train_loader, train_dataset = createDataLoaderAndDataSet(formated_full_gt_df, 
                                                         'test', 
                                                         imgsz=IMG_SIZE, 
                                                         batch_size=20, 
                                                         augment=True)

img, labels_out, filepath, shapes = train_dataset[2190 ]

imgNT = img.numpy().transpose(1, 2, 0).astype(np.uint8).copy() #, cv2.COLOR_BGR2RGB)
print(labels_out)
for coord in labels_out[:, 2:]:
    # print(coord)
    h, w = shapes[0]
    xywh = UnmakeRel(coord, IMG_SIZE, IMG_SIZE)
    x1y1x2y2 = ConvertCenterXYWH2CV2Rectangle(xywh)
    print(x1y1x2y2)
    imgNT = cv2.rectangle(imgNT, (x1y1x2y2[0], x1y1x2y2[1]), 
                        (x1y1x2y2[2], x1y1x2y2[3]), 
                        (255, 0, 0), 
                        3)
    
plt.imshow(imgNT)

In [ ]:
model_cfg_file = YOLO_MODEL_HOME_DIR / 'models/yolov5l_custom_anchors.yaml'
model = Model(cfg=model_cfg_file, ch=3, nc=1)

In [ ]:
from tqdm import tqdm

start_epoch = 0
epochs = 5

nb = len(train_loader)  # number of batches

for epoch in range(start_epoch, epochs):
    model.train()
    mloss = torch.zeros(3, device=device)  # mean losses
    pbar = enumerate(train_loader)
    pbar = tqdm(pbar, total=nb, bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')  # progress bar
    optimizer.zero_grad()
    nw = max(round(hyp['warmup_epochs'] * nb), 1000)  # number of warmup iterations, max(3 epochs, 1k iterations)
        
    for i, (imgs, targets, paths, _) in pbar:
        ni = i + nb * epoch  # number integrated batches (since train start)
        # Warmup
        if ni <= nw:
            xi = [0, nw]  # x interp
            # compute_loss.gr = np.interp(ni, xi, [0.0, 1.0])  # iou loss ratio (obj_loss = 1.0 or iou)
            accumulate = max(1, np.interp(ni, xi, [1, nbs / batch_size]).round())
            for j, x in enumerate(optimizer.param_groups):
                # bias lr falls from 0.1 to lr0, all other lrs rise from 0.0 to lr0
                x['lr'] = np.interp(ni, xi, [hyp['warmup_bias_lr'] if j == 2 else 0.0, x['initial_lr'] * lf(epoch)])
                if 'momentum' in x:
                    x['momentum'] = np.interp(ni, xi, [hyp['warmup_momentum'], hyp['momentum']])    

In [ ]:
pbar = enumerate(train_loader)
for i, (imgs, targets, paths, _) in pbar:
    print(i)
    pass
print('completed')

In [ ]:
train_dataset[15466]

In [ ]:
# Optimizer
batch_size = 1
nbs = 64  # nominal batch size
accumulate = max(round(nbs / batch_size), 1)  # accumulate loss before optimizing
hyp['weight_decay'] *= batch_size * accumulate / nbs  # scale weight_decay

g0, g1, g2 = [], [], []  # optimizer parameter groups
for v in model.modules():
    if hasattr(v, 'bias') and isinstance(v.bias, nn.Parameter):  # bias
        g2.append(v.bias)
    if isinstance(v, nn.BatchNorm2d):  # weight (no decay)
        g0.append(v.weight)
    elif hasattr(v, 'weight') and isinstance(v.weight, nn.Parameter):  # weight (with decay)
        g1.append(v.weight)

optimizer = 'AdamW'
if optimizer == 'Adam':
    optimizer = Adam(g0, lr=hyp['lr0'], betas=(hyp['momentum'], 0.999))  # adjust beta1 to momentum
elif optimizer == 'AdamW':
    optimizer = AdamW(g0, lr=hyp['lr0'], betas=(hyp['momentum'], 0.999))  # adjust beta1 to momentum
else:
    optimizer = SGD(g0, lr=hyp['lr0'], momentum=hyp['momentum'], nesterov=True)
        
optimizer.add_param_group({'params': g1, 'weight_decay': hyp['weight_decay']})  # add g1 with weight_decay
optimizer.add_param_group({'params': g2})  # add g2 (biases)

In [ ]:
m.anchors
# 7,12,  10,17,  13,24,  26,20,  17,32,  23,41,  31,54,  41,72,  58,100 # custom anchors

In [ ]:
 t_ = torch.tensor([[[ 10.,  13.],
         [ 16.,  30.],
         [ 33.,  23.]],  # P3/8-small

        [[ 30.,  61.],
         [ 62.,  45.],
         [ 59., 119.]],  # P4/16-medium

        [[116.,  90.],
         [156., 198.],
         [373., 326.]]], dtype=torch.float16) 

In [ ]:
Model